In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp_api
import kerastuner
import numpy as np
import pandas as pd
import os
import json
import datetime
import dill

In [2]:
from tensorflow.keras.layers import (
    Dense, 
    Dropout,
    LSTMCell,
    RNN,
    StackedRNNCells
)

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
dataset_name = "SEG_AR_variants"
variants = "single_stacked"

In [5]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + variants + "/" + timestamp
version_dir = "version/" + variants + "/" + timestamp 

os.makedirs(log_dir)
os.makedirs(version_dir)
timestamp

'20200921-215749'

In [5]:
static_params = {
    'PAST_HISTORY': 16,
    'FUTURE_TARGET': 8,
    'BATCH_SIZE': 512,
    'BUFFER_SIZE': 200000,
    'EPOCHS': 500,
    'VOCAB_SIZE': 16293
 }

In [6]:
hparams = {
    "HP_LSTM_1_UNITS" : 128,
    "HP_LSTM_2_UNITS" : 128,
    "HP_LSTM_1_DROPOUT" : 0.5,
    "HP_LSTM_2_DROPOUT" : 0.5,
    "HP_LEARNING_RATE" : 1e-3,
}

In [7]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [8]:
train_set = np.genfromtxt("data/SEG_train_set.csv", delimiter="\n", dtype=np.int32)
x_train, y_train = generate_timeseries(train_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().batch(static_params["BATCH_SIZE"]).shuffle(static_params["BUFFER_SIZE"])

In [9]:
val_set = np.genfromtxt("data/SEG_val_set.csv", delimiter="\n", dtype=np.int32)
x_val, y_val = generate_timeseries(val_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.cache().batch(static_params["BATCH_SIZE"])

In [10]:
class SEGARStacked(keras.Model):
    def __init__(self, units_1, units_2, dropout_1, dropout_2, output_steps, output_size):
        super().__init__()
        self.output_steps = output_steps
        self.units_1 = units_1
        self.units_2 = units_2
        self.lstm_cells = [
            LSTMCell(units_1, dropout=dropout_1),
            LSTMCell(units_2, dropout=dropout_2),
        ]
        self.lstm_stack = StackedRNNCells(self.lstm_cells)

        self.lstm_rnn = RNN(self.lstm_stack, return_state=True)
        self.dense = Dense(output_size, activation="softmax")

    @tf.function
    def warmup(self, inputs):
        onehot_inputs = tf.squeeze(tf.one_hot(inputs, static_params["VOCAB_SIZE"]), axis=2)

        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(onehot_inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)

        return prediction, state

    @tf.function
    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        #predictions = []
        predictions = tf.TensorArray(tf.float32, size=self.output_steps, clear_after_read=False)
        # Initialize the lstm state
        prediction, state = self.warmup(inputs)

        # Insert the first prediction
        #predictions.append(prediction)
        predictions = predictions.write(0, prediction)

        # Run the rest of the prediction steps
        for i in tf.range(1, self.output_steps):
            # Use the last prediction as input.
            x = prediction

            # Execute one lstm step.
            x, state = self.lstm_stack(x, states=state, training=training)
            state = list(state)     # output state of lstm_stack is tuple => chnage to list

            # Convert the lstm output to a prediction.
            prediction = self.dense(x)

            # Add the prediction to the output
            #predictions.append(prediction)
            predictions = predictions.write(i, prediction)

        # predictions.shape => (time, batch, features)
        #predictions = tf.stack(predictions)
        predictions = predictions.stack()

        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])

        return predictions

In [11]:
model = SEGARStacked(
    units_1=hparams["HP_LSTM_1_UNITS"], units_2=hparams["HP_LSTM_2_UNITS"], dropout_1=hparams["HP_LSTM_1_DROPOUT"], 
    dropout_2=hparams["HP_LSTM_2_DROPOUT"], output_steps=static_params["FUTURE_TARGET"], output_size=static_params["VOCAB_SIZE"])

In [12]:
model.call(x_train[0].reshape(1, -1, 1))

<tf.Tensor: shape=(1, 8, 16293), dtype=float32, numpy=
array([[[6.1452862e-05, 6.1393454e-05, 6.1378698e-05, ...,
         6.1322506e-05, 6.1431478e-05, 6.1523453e-05],
        [6.1451829e-05, 6.1396138e-05, 6.1378036e-05, ...,
         6.1320206e-05, 6.1428480e-05, 6.1518585e-05],
        [6.1448605e-05, 6.1399063e-05, 6.1377206e-05, ...,
         6.1319079e-05, 6.1423423e-05, 6.1508836e-05],
        ...,
        [6.1431805e-05, 6.1405735e-05, 6.1374187e-05, ...,
         6.1324310e-05, 6.1404418e-05, 6.1469669e-05],
        [6.1425351e-05, 6.1406667e-05, 6.1373146e-05, ...,
         6.1328130e-05, 6.1398452e-05, 6.1456878e-05],
        [6.1418992e-05, 6.1406899e-05, 6.1372157e-05, ...,
         6.1332437e-05, 6.1393082e-05, 6.1445084e-05]]], dtype=float32)>

In [13]:
model.compile(
    optimizer=keras.optimizers.Nadam(hparams["HP_LEARNING_RATE"]),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [74]:
with open("static/test_pipeline.pkl", "rb") as p:
    test_pipeline = dill.load(p)

test_set = np.genfromtxt("data/SEG_test_set_original.csv", delimiter="\n", dtype=np.int64)
processed_test_set = test_pipeline.transform(test_set.copy())
x_test, y_test = generate_timeseries(processed_test_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])

In [90]:
with tf.summary.create_file_writer(log_dir).as_default():
    hp_api.hparams(hparams)
    history = model.fit(train_data, validation_data=val_data, epochs=1, callbacks=[
        keras.callbacks.EarlyStopping('val_accuracy', patience=5),
        keras.callbacks.TensorBoard(log_dir)
        ])

    #loss, acc = model.evaluate(x_test, y_test)
    #tf.summary.scalar("test_loss", loss, step=1)
    #tf.summary.scalar("test_accuracy", acc, step=1)

1/315 [..............................] - ETA: 0s - loss: 9.6986 - accuracy: 0.0000e+  2/315 [..............................] - ETA: 36:42 - loss: 9.6982 - accuracy: 0.0000e+0  3/315 [..............................] - ETA: 48:59 - loss: 9.6965 - accuracy: 0.140  4/315 [..............................] - ETA: 54:36 - loss: 9.6969 - accuracy: 0.1214

KeyboardInterrupt: 

In [76]:
new_model = tf.saved_model.load("version/20200918-103208")

OSError: SavedModel file does not exist at: version/20200918-103208/{saved_model.pbtxt|saved_model.pb}

In [77]:
new_model.summary()

NameError: name 'new_model' is not defined

In [78]:
tf.saved_model.save(model, version_dir, 
    signatures=model.call.get_concrete_function(tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32, name="call")))

INFO:tensorflow:Assets written to: version/20200918-142458\assets
